In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model

import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from imageai.Detection import ObjectDetection
from IPython.display import clear_output
from sklearn import preprocessing
import pickle
from mpl_toolkits.axes_grid1 import Grid

# example of converting an image with the Keras API
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import math

/home/csgrads/samad028/anaconda3/envs/py37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/csgrads/samad028/anaconda3/envs/py37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/csgrads/samad028/anaconda3/envs/py37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/csgrads/samad028

In [2]:
%run paths.py # Load all our paths
%run methods.py # Load all our methods

detector_name = "yolo-30" # earlier "yolo"
DETECTIONS_FILE = os.path.join(DETECTIONS_DIR , detector_name+"-detections.csv")

os.listdir(DATASET_DIR)

['test_images',
 'train_masks.zip',
 'train_images.zip',
 'car_models',
 '.ipynb_checkpoints',
 'car_models_json.zip',
 'personal',
 'car_models.zip',
 'test_masks',
 'train.csv',
 'train_images',
 'test_images.zip',
 'train_masks',
 'camera.zip',
 'sample_submission.csv',
 'car_models_json',
 'test_masks.zip',
 'camera']

In [3]:
df_detections = pd.read_csv(DETECTIONS_FILE)
#df_detections.drop_duplicates(keep = "first", inplace = True)
#df_detections.to_csv(DETECTIONS_FILE)
display(df_detections)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,ImageId,Probability,x1,x2,y1,y2,id,yaw,pitch,roll,x,y,z
0,0,0,0,ID_8a6e65317-1,64.780223,1949.0,2172.0,1740.0,1847.0,70.0,0.163072,-1.56865,-3.11754,10.390000,11.22190,59.7825
1,1,1,1,ID_8a6e65317-2,98.063952,2522.0,3050.0,1807.0,2039.0,56.0,0.181647,-1.46947,-3.12159,9.603320,4.66632,19.3390
2,2,2,2,ID_8a6e65317-3,98.919398,2873.0,3384.0,1839.0,2147.0,16.0,0.254839,-2.57534,-3.10256,7.965390,3.20066,11.0225
3,3,3,3,ID_8a6e65317-4,99.524534,607.0,941.0,1727.0,1929.0,70.0,0.141942,-3.13950,3.11969,-9.592360,5.13662,24.7337
4,4,4,4,ID_337ddc495-2,65.653896,1303.0,1397.0,1754.0,1831.0,28.0,0.186114,0.35728,-3.10043,-7.498180,10.31120,54.0320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31474,31474,31474,31474,ID_2dee4d6e6-1,37.749365,1680.0,1747.0,1697.0,1752.0,46.0,0.087367,-0.09370,-3.11728,0.248309,22.63240,134.5110
31475,31475,31475,31475,ID_2dee4d6e6-3,45.299059,1028.0,1116.0,1706.0,1750.0,28.0,0.159466,2.70411,3.11200,-22.109500,14.78060,89.1042
31476,31476,31476,31476,ID_2dee4d6e6-4,46.475521,959.0,1049.0,1710.0,1753.0,35.0,0.143199,-3.01809,-3.13069,-20.370000,11.28470,66.8392
31477,31477,31477,31477,ID_2dee4d6e6-6,76.999432,584.0,820.0,1710.0,1826.0,35.0,0.166939,3.08081,-3.11637,-14.889200,6.55844,35.4492


In [4]:
"""imgSizes = []

for index,row in df_detections.iterrows():
    
    ImageIdSubId = row["ImageId"]
    ImageIdSubIdList = ImageIdSubId.split("-")
    ImageId = ImageIdSubIdList[0]
    SubId = ImageIdSubIdList[1]
    imagePath = os.path.join(DATASET_DIR , "detections/"+detector_name+"/"+ImageId+".jpg-objects/car-"+SubId+".jpg")
    
    img = open_image(imagePath)
    dimensions = img.shape
    heightImage, widthImage, channels = dimensions
    
    imgSizes.append(heightImage)
    
    clear_output(wait=True)
    print(f"{index+1} Images Analysed. So far, Minimum: {min(imgSizes)}, Maximum: {max(imgSizes)}")
    
    #visualize_image(imagePath,width=5,title=ImageIdSubId)
    
    break"""

#335504 Images Analysed. So far, Minimum: 45, Maximum: 1190
pass

In [5]:
#df_temp = df_detections.head(10000)
#df_temp = df_detections.sample(1000000)
df_temp = df_detections.copy() # Take Whole Dataset

totalDetectionsRows = len(df_temp.index)

train_x = []
train_y = []

countedDetectionRows = 0

for index,row in df_temp.iterrows():
    
    countedDetectionRows +=1
    
    ImageIdSubId = row["ImageId"]
    
    extra_inputs = [row["x1"],row["y1"],row["x2"],row["y2"]]
    temp_array = format_inputs_appended_with_image(ImageIdSubId,extra_inputs,detector_name)
    
    train_x.append( temp_array )
    train_y.append([row["x"],row["y"],row["z"],row["yaw"],row["pitch"],row["roll"]])
    
    if(countedDetectionRows%10)==0 or countedDetectionRows==totalDetectionsRows:
        clear_output(wait=True)
        print(f"{countedDetectionRows}/{totalDetectionsRows} Completed. Just tested ImageID-SubID: {ImageIdSubId}")
    
train_x = np.array(train_x)
train_y = np.array(train_y)
df_temp

31479/31479 Completed. Just tested ImageID-SubID: ID_2dee4d6e6-7


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,ImageId,Probability,x1,x2,y1,y2,id,yaw,pitch,roll,x,y,z
0,0,0,0,ID_8a6e65317-1,64.780223,1949.0,2172.0,1740.0,1847.0,70.0,0.163072,-1.56865,-3.11754,10.390000,11.22190,59.7825
1,1,1,1,ID_8a6e65317-2,98.063952,2522.0,3050.0,1807.0,2039.0,56.0,0.181647,-1.46947,-3.12159,9.603320,4.66632,19.3390
2,2,2,2,ID_8a6e65317-3,98.919398,2873.0,3384.0,1839.0,2147.0,16.0,0.254839,-2.57534,-3.10256,7.965390,3.20066,11.0225
3,3,3,3,ID_8a6e65317-4,99.524534,607.0,941.0,1727.0,1929.0,70.0,0.141942,-3.13950,3.11969,-9.592360,5.13662,24.7337
4,4,4,4,ID_337ddc495-2,65.653896,1303.0,1397.0,1754.0,1831.0,28.0,0.186114,0.35728,-3.10043,-7.498180,10.31120,54.0320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31474,31474,31474,31474,ID_2dee4d6e6-1,37.749365,1680.0,1747.0,1697.0,1752.0,46.0,0.087367,-0.09370,-3.11728,0.248309,22.63240,134.5110
31475,31475,31475,31475,ID_2dee4d6e6-3,45.299059,1028.0,1116.0,1706.0,1750.0,28.0,0.159466,2.70411,3.11200,-22.109500,14.78060,89.1042
31476,31476,31476,31476,ID_2dee4d6e6-4,46.475521,959.0,1049.0,1710.0,1753.0,35.0,0.143199,-3.01809,-3.13069,-20.370000,11.28470,66.8392
31477,31477,31477,31477,ID_2dee4d6e6-6,76.999432,584.0,820.0,1710.0,1826.0,35.0,0.166939,3.08081,-3.11637,-14.889200,6.55844,35.4492


In [6]:
"""
>>> np.savez_compressed('/tmp/123', a=test_array, b=test_vector)
>>> loaded = np.load('/tmp/123.npz')
>>> print(np.array_equal(test_array, loaded['a']))
True
"""
compressedFilename = os.path.join(DETECTIONS_DIR , f"training-set-{countedDetectionRows}-{detector_name}.npz")

In [7]:
np.savez_compressed(compressedFilename, x=train_x, y=train_y) # Comment to avoid unintended save
print(f"Compressed File saved at {compressedFilename}")

Compressed File saved at ../detections/training-set-31479-yolo-30.npz


In [8]:
# Loads in less than 5 mins
train = np.load(compressedFilename)
train_x = train['x']
train_y = train['y']
print(f"Compressed File Loaded. train_x shape: {train_x.shape}")

Compressed File Loaded. train_x shape: (31479, 48, 48, 4)


In [9]:
IMG_HEIGHT, IMG_WIDTH = 48,48
INPUT_SHAPE = train_x.shape[1:]
print(INPUT_SHAPE)

"""train_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data

train_data_gen = train_image_generator.flow_from_dataframe(dataframe=df_temp,
                                                           x_col='filename',
                                                           y_col=["x","y","z","yaw","pitch","roll"],                                                           
                                                           #y_col="roll",                                                           
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='raw')"""

pass

(48, 48, 4)


In [10]:
def binary_form_output_dim(i,startWith = 256):
    num = i
    
    if num < 1:
        num = 1
        
    while((startWith/num)%2!=0):
        num -= 1
    
    if num < 1:
        num = 1
        
    number = startWith/num
    return int(number)

def return_cnn_model(input_shape=INPUT_SHAPE, cnn_layers=2, mlp_layers=1, cnn_act='relu', mlp_act='linear'):

    model = models.Sequential()
    model.add(layers.Conv2D(256, (3, 3), activation=cnn_act, input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    
    for i in range(cnn_layers):
        op_units = binary_form_output_dim(i,startWith = 256)
        model.add(layers.Conv2D(op_units, (3, 3), activation=cnn_act))
        model.add(layers.MaxPooling2D((2, 2)))
        
    #model.add(layers.Conv2D(64, (3, 3), activation='relu'))

    model.add(layers.Flatten())
    
    for i in range(mlp_layers):
        op_units = binary_form_output_dim(i,startWith = 256)
        model.add(layers.Dense(op_units, activation=mlp_act))
    
    model.add(layers.Dense(6))

    # model.summary()
    plot_model(model, to_file=f"analysis/models/model_cnn-{cnn_layers}-{cnn_act}_mlp-{mlp_layers}-{mlp_act}.png", show_shapes=True, show_layer_names=False)
    
    return model

In [11]:
epochs = 1000

cnn_layers_opt = [3,6,9] # [1,2,3,4]
mlp_layers_opt = [3,6,9] # [1,2,3,4]
cnn_act_opt = ['relu','linear','sigmoid'] # ['relu','linear','sigmoid']
mlp_act_opt = ['relu','linear','sigmoid'] # ['relu','linear','sigmoid']

totalGraphsCount = len(cnn_layers_opt) * len(mlp_layers_opt) * len(cnn_act_opt) * len(mlp_act_opt)
graphsDimension1 = int(round(math.sqrt(totalGraphsCount),0))
graphsDimension2 = int(round(totalGraphsCount / graphsDimension1,0))

history = {}
model = {}

for cnn_layers in cnn_layers_opt:
    for mlp_layers in mlp_layers_opt:
        for cnn_act in cnn_act_opt:
            for mlp_act in mlp_act_opt:
                
                ID = f"c{cnn_layers}-{cnn_act[0]}-m{mlp_layers}-{mlp_act[0]}-on-{detector_name}"
                #ID = f"cnn-{cnn_layers}-{cnn_act}_mlp-{mlp_layers}-{mlp_act}_epoch-{epochs}"
                print(f"Running {ID}")

                model[ID] = return_cnn_model(cnn_layers=cnn_layers, mlp_layers=mlp_layers, cnn_act=cnn_act, mlp_act=mlp_act)
                model[ID].compile(optimizer='adam',
                              loss='mean_squared_error',
                              metrics=['mse','accuracy','mean_absolute_percentage_error','mean_absolute_error'])
                
                
                
                history[ID] = model[ID].fit(x=train_x,y=train_y,epochs=epochs,
                                        use_multiprocessing=True,
                                        verbose=0,validation_split=0.1)

                model[ID].save(os.path.join(MODELS_DIR, f'model_{ID}.h5'))

Running c3-r-m3-r-on-yolo-30
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Running c3-r-m3-l-on-yolo-30
Running c3-r-m3-s-on-yolo-30
Running c3-l-m3-r-on-yolo-30
Running c3-l-m3-l-on-yolo-30
Running c3-l-m3-s-on-yolo-30
Running c3-s-m3-r-on-yolo-30
Running c3-s-m3-l-on-yolo-30
Running c3-s-m3-s-on-yolo-30
Running c3-r-m6-r-on-yolo-30
Running c3-r-m6-l-on-yolo-30
Running c3-r-m6-s-on-yolo-30
Running c3-l-m6-r-on-yolo-30
Running c3-l-m6-l-on-yolo-30
Running c3-l-m6-s-on-yolo-30
Running c3-s-m6-r-on-yolo-30
Running c3-s-m6-l-on-yolo-30
Running c3-s-m6-s-on-yolo-30
Running c3-r-m9-r-on-yolo-30
Running c3-r-m9-l-on-yolo-30
Running c3-r-m9-s-on-yolo-30
Running c3-l-m9-r-on-yolo-30
Running c3-l-m9-l-on-yolo-30
Running c3-l-m9-s-on-yolo-30
Running c3-s-m9-r-on-yolo-30
Running c3-s-m9-l-on-yolo-30
Running c3-s-m9-s-on-yolo-30
Running c6-r-m3-r-on-yolo-30


ValueError: Negative dimension size caused by subtracting 3 from 1 for 'conv2d_112/Conv2D' (op: 'Conv2D') with input shapes: [?,1,1,128], [3,3,128,128].

In [ ]:
def save_history(history,historyFilename):
    historySave = dict()
    for i,his in history.items():
        historySave[i] = his.history
    with open(historyFilename, 'wb') as handle:
        pickle.dump(historySave, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
historyFilename = f"history/history-{epochs}-configs-{totalGraphsCount}-on-{detector_name}.pickle"

In [ ]:
save_history(history,historyFilename)

In [ ]:
with open(historyFilename, 'rb') as handle:
    historyUnPacked = pickle.load(handle)

In [ ]:
linestyles = ['-','--','-.',':']
markerstyles = ['*','s','v','o','<','p','^','.','>']

plt.close('all')
fig = plt.figure()
fig.set_figheight( int(round(totalGraphsCount,0)) * 5 )
fig.set_figwidth( int(round(totalGraphsCount,0)) * 5 )
grid = Grid(fig, rect=111, nrows_ncols=(graphsDimension1, graphsDimension2), axes_pad=0.3, label_mode='L',)

gridCount = 0

for cnn_layers in cnn_layers_opt:
    for mlp_layers in mlp_layers_opt:
        for cnn_act in cnn_act_opt:
            for mlp_act in mlp_act_opt:
                
                # ID = f"cnn-{cnn_layers}-{cnn_act}_mlp-{mlp_layers}-{mlp_act}_epoch-{epochs}"                
                ID = f"c{cnn_layers}-{cnn_act[0]}-m{mlp_layers}-{mlp_act[0]}-on-{detector_name}"
                
                print(f"Running {ID}")
                
                for index,row in df_temp.sample(50).iterrows():
    
                    ImageIdSubId = row["ImageId"]

                    extra_inputs = [row["x1"],row["y1"],row["x2"],row["y2"]]
                    temp_array = format_inputs_appended_with_image(ImageIdSubId,extra_inputs,detector_name)

                    temp_array = np.array([temp_array])

                    print(model[ID].predict(temp_array))
                    print(row["x"],row["y"],row["z"],row["yaw"],row["pitch"],row["roll"])

                    break
                
                #fig, ax = plt.subplots()
                ax = grid[gridCount]
                gridCount += 1
                
                count = 0

                ax.plot(historyUnPacked[ID]['mean_squared_error'], 
                            label="Training", 
                            linestyle=linestyles[count%len(linestyles)], 
                            marker=markerstyles[count%len(markerstyles)])
                    
                count+=1

                ax.plot(historyUnPacked[ID]['val_mean_squared_error'], 
                        label="Validation", 
                        linestyle=linestyles[count%len(linestyles)], 
                        marker=markerstyles[count%len(markerstyles)])
                    
                    
                ax.set(xlabel='Epochs', 
                       ylabel='MSE',
                       title=ID)
                ax.grid()
                
                ax.legend(loc='upper right')
                
                #ax.title.set_visible(False)
                #plt.legend(loc='lower left')
                #filename = os.path.join(ANALYSIS_DIR, f"training-{ID}")   
                #fig.savefig(filename)
                #plt.show()
                #plt.tight_layout()
            
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
filename = os.path.join(ANALYSIS_DIR, f"grid-MSE-epochs-{epochs}")   
fig.savefig(filename)

In [24]:
for index,row in df_temp.sample(50).iterrows():
    
    ImageIdSubId = row["ImageId"]
    
    extra_inputs = [row["x1"],row["y1"],row["x2"],row["y2"]]
    temp_array = format_inputs_appended_with_image(ImageIdSubId,extra_inputs,detector_name)
    
    temp_array = np.array([temp_array])
    
    print(model.predict(temp_array))
    print(row["x"],row["y"],row["z"],row["yaw"],row["pitch"],row["roll"])
    
    break

[[ 6.4015408   4.7000833  19.280918    0.10241883  0.02203971 -2.2503724 ]]
6.21709 5.9477400000000005 22.5135 0.15919 0.0125043 -3.09076


In [ ]:
#from tensorflow.keras.initializers import glorot_uniform
#model = tf.keras.models.load_model('cnn-1.h5',custom_objects={'GlorotUniform': glorot_uniform()})

In [ ]:
# Yet to implement the following properly as a Keras Metric (Mean absolute Precision)


from math import sqrt, acos, pi, sin, cos
from scipy.spatial.transform import Rotation as R
from sklearn.metrics import average_precision_score
from multiprocessing import Pool

def expand_df(df, PredictionStringCols):
    df = df.dropna().copy()
    df['NumCars'] = [int((x.count(' ')+1)/7) for x in df['PredictionString']]

    image_id_expanded = [item for item, count in zip(df['ImageId'], df['NumCars']) for i in range(count)]
    prediction_strings_expanded = df['PredictionString'].str.split(' ',expand = True).values.reshape(-1,7).astype(float)
    prediction_strings_expanded = prediction_strings_expanded[~np.isnan(prediction_strings_expanded).all(axis=1)]
    df = pd.DataFrame(
        {
            'ImageId': image_id_expanded,
            PredictionStringCols[0]:prediction_strings_expanded[:,0],
            PredictionStringCols[1]:prediction_strings_expanded[:,1],
            PredictionStringCols[2]:prediction_strings_expanded[:,2],
            PredictionStringCols[3]:prediction_strings_expanded[:,3],
            PredictionStringCols[4]:prediction_strings_expanded[:,4],
            PredictionStringCols[5]:prediction_strings_expanded[:,5],
            PredictionStringCols[6]:prediction_strings_expanded[:,6]
        })
    return df

def TranslationDistance(p,g, abs_dist = False):
    dx = p['x'] - g['x']
    dy = p['y'] - g['y']
    dz = p['z'] - g['z']
    diff0 = (g['x']**2 + g['y']**2 + g['z']**2)**0.5
    diff1 = (dx**2 + dy**2 + dz**2)**0.5
    if abs_dist:
        diff = diff1
    else:
        diff = diff1/diff0
    return diff

def RotationDistance(p, g):
    true=[ g['pitch'] ,g['yaw'] ,g['roll'] ]
    pred=[ p['pitch'] ,p['yaw'] ,p['roll'] ]
    q1 = R.from_euler('xyz', true)
    q2 = R.from_euler('xyz', pred)
    diff = R.inv(q2) * q1
    W = np.clip(diff.as_quat()[-1], -1., 1.)
    
    # in the official metrics code:
    # https://www.kaggle.com/c/pku-autonomous-driving/overview/evaluation
    #   return Object3D.RadianToDegree( Math.Acos(diff.W) )
    # this code treat θ and θ+2π differntly.
    # So this should be fixed as follows.
    W = (acos(W)*360)/pi
    if W > 180:
        W = 360 - W
    return W

thres_tr_list = [0.1, 0.09, 0.08, 0.07, 0.06, 0.05, 0.04, 0.03, 0.02, 0.01]
thres_ro_list = [50, 45, 40, 35, 30, 25, 20, 15, 10, 5]

def check_match(idx):
    keep_gt=False
    thre_tr_dist = thres_tr_list[idx]
    thre_ro_dist = thres_ro_list[idx]
    train_dict = {imgID:str2coords(s, names=['carid_or_score', 'pitch', 'yaw', 'roll', 'x', 'y', 'z']) for imgID,s in zip(train_df['ImageId'],train_df['PredictionString'])}
    valid_dict = {imgID:str2coords(s, names=['pitch', 'yaw', 'roll', 'x', 'y', 'z', 'carid_or_score']) for imgID,s in zip(valid_df['ImageId'],valid_df['PredictionString'])}
    result_flg = [] # 1 for TP, 0 for FP
    scores = []
    MAX_VAL = 10**10
    for img_id in valid_dict:
        for pcar in sorted(valid_dict[img_id], key=lambda x: -x['carid_or_score']):
            # find nearest GT
            min_tr_dist = MAX_VAL
            min_idx = -1
            for idx, gcar in enumerate(train_dict[img_id]):
                tr_dist = TranslationDistance(pcar,gcar)
                if tr_dist < min_tr_dist:
                    min_tr_dist = tr_dist
                    min_ro_dist = RotationDistance(pcar,gcar)
                    min_idx = idx
                    
            # set the result
            if min_tr_dist < thre_tr_dist and min_ro_dist < thre_ro_dist:
                if not keep_gt:
                    train_dict[img_id].pop(min_idx)
                result_flg.append(1)
            else:
                result_flg.append(0)
            scores.append(pcar['carid_or_score'])
    
    return result_flg, scores


ap = average_precision_score(result_flg, scores)*recall